In [7]:
import mip
from collections import namedtuple, defaultdict
from itertools import product


In [8]:
def bin_packing(A, B, c, sizes):
    # c capacity of all the bins
    
    model = mip.Model()
    
    x = {(a, b): model.add_var(f"x_{a}_{b}", obj=0, var_type=mip.BINARY) for a, b in product(A, B)}
    y = {b: model.add_var(f"y_{b}", obj=1, var_type=mip.BINARY) for b in B}
    
    for b in B:
        model.add_constr(mip.quicksum(sizes[a] * x[a, b] for a in A) <= c * y[b])
    
    for a in A:
        model.add_constr(mip.quicksum(x[a, b] for b in B) == 1)
    
    model.optimize()
    
    return {k for k, v in x.items() if v.x > 0}, {k for k, v in y.items() if v.x > 0}

bin_packing({"a", "b", "c"}, {1, 2, 3}, 15, {"a": 10, "b": 4, "c": 11} )


InterfacingError: Gurobi environment could not be loaded, check your license.

In [ ]:
def bin_packing_frac(A, B, c, sizes):
    # c capacity of all the bins
    
    model = mip.Model()
    
    x = {(a, b): model.add_var(f"x_{a}_{b}", obj=0, var_type=mip.BINARY) for a, b in product(A, B)}
    y = {b: model.add_var(f"y_{b}", obj=1, var_type=mip.BINARY) for b in B}
    
    lambd = {(a, b): model.add_var(f"lambda_{a}_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for a, b in product(A, B)}
    alpha = {b: model.add_var(f"alpha_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for b in B}
    
    for b in B:
        model.add_constr(mip.quicksum(sizes[a] * lambd[a, b] for a in A) <= c * y[b])
    
    for a in A:
        model.add_constr(mip.quicksum(lambd[a, b] for b in B) == 1)
    
    for a, b in product(A, B):
        # non-linear constraint `model.add_constr(lambd[a, b] == alpha[b] * x[a, b])` is remplaced by:

        model.add_constr(lambd[a, b] <= x[a, b])
        model.add_constr(lambd[a, b] <= alpha[b])
        model.add_constr(lambd[a, b] + 1 >= alpha[b] + x[a, b])
    
    
    
    model.optimize()
    
    print(model.status)
    
    return {k for k, v in x.items() if v.x > 0}, {k for k, v in y.items() if v.x > 0}

bin_packing_frac({"a", "b", "c"}, {1, 2, 3}, 15, {"a": 32, "b": 4, "c": 4} )


In [3]:
def bin_packing_frac_balanced(A, B, c, sizes, k):
    # c capacity of all the bins
    
    model = mip.Model()
    
    x = {(a, b): model.add_var(f"x_{a}_{b}", obj=0, var_type=mip.BINARY) for a, b in product(A, B)}
    y = {b: model.add_var(f"y_{b}", obj=1, var_type=mip.BINARY) for b in B}
    
    lambd = {(a, b): model.add_var(f"lambda_{a}_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for a, b in product(A, B)}
    alpha = {b: model.add_var(f"alpha_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for b in B}
    
    for b in B:
        model.add_constr(mip.quicksum(sizes[a] * lambd[a, b] for a in A) <= c * y[b])
    
    for a in A:
        model.add_constr(mip.quicksum(lambd[a, b] for b in B) == 1)
    
    for a, b in product(A, B):

        # non-linear constraint `model.add_constr(lambd[a, b] == alpha[b] * x[a, b])` is remplaced by:
        model.add_constr(lambd[a, b] <= x[a, b])
        model.add_constr(lambd[a, b] <= alpha[b])
        model.add_constr(lambd[a, b] + 1 >= alpha[b] + x[a, b])
    
    for b in B:
        model.add_constr(mip.quicksum(x[a, b] for a in A) <= k)
    
    
    
    model.optimize()
    
    print(model.status)
    
    return {k for k, v in x.items() if v.x > 0}, {k for k, v in y.items() if v.x > 0}

bin_packing_frac_balanced({"a", "b", "c"}, range(30), 15, {"a": 33, "b": 4, "c": 4}, 2)


InterfacingError: Gurobi environment could not be loaded, check your license.

# Balanced bin packing

Every input can be split at most k times to fit the bins.

In [52]:
def bin_packing_frac_balanced(A, B, k):
    # c capacity of all the bins
    
    model = mip.Model()
    
    x = {(a, b): model.add_var(f"x_{a}_{b}", obj=1, var_type=mip.BINARY) for a, b in product(A, B)}
    y = {b: model.add_var(f"y_{b}", obj=1000, var_type=mip.BINARY) for b in B}
    
    lambd = {(a, b): model.add_var(f"lambda_{a}_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for a, b in product(A, B)}
    alpha = {b: model.add_var(f"alpha_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for b in B}
    
    for b in B:
        model.add_constr(mip.quicksum(A[a] * lambd[a, b] for a in A) <= B[b] * y[b])
    
    for a in A:
        model.add_constr(mip.quicksum(lambd[a, b] for b in B) == 1)
    
    for a, b in product(A, B):

        # non-linear constraint `model.add_constr(lambd[a, b] == alpha[b] * x[a, b])` is remplaced by:
        model.add_constr(lambd[a, b] <= x[a, b])
        model.add_constr(lambd[a, b] <= alpha[b])
        model.add_constr(lambd[a, b] + 1 >= alpha[b] + x[a, b])
    
    for b in B:
        model.add_constr(mip.quicksum(x[a, b] for a in A) <= k)

    model.optimize()
    
    print(model.status)
    
    return {k for k, v in x.items() if v.x > 0}, {k for k, v in y.items() if v.x > 0}


_, l = bin_packing_frac_balanced({"a": 15, "b": 15, "c": 15}, {i: 15 for i in range(10)}, 2)
bin_packing_frac_balanced({"a": 20, "b": 15, "c": 5}, {i: 15 for i in range(len(l))}, 2)



OptimizationStatus.OPTIMAL
OptimizationStatus.OPTIMAL


({('a', 0), ('a', 2), ('b', 1), ('c', 0), ('c', 2)}, {0, 1, 2})

In [ ]:
def bin_packing_frac_balanced_2steps(A, B, C, k):
    # c capacity of all the bins
    
    model = mip.Model()
    
    x1 = {(a, b): model.add_var(f"x1_{a}_{b}", obj=1, var_type=mip.BINARY) for a, b in product(A, B)}
    x2 = {(b, c): model.add_var(f"x2_{b}_{c}", obj=1, var_type=mip.BINARY) for b, c in product(B, C)}
    y = {b: model.add_var(f"y_{b}", obj=1000, var_type=mip.BINARY) for b in B}
    
    lambd1 = {(a, b): model.add_var(f"lambda1_{a}_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for a, b in product(A, B)}
    lambd2 = {(b, c): model.add_var(f"lambda2_{b}_{c}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for b, c in product(B, C)}
    alpha1 = {b: model.add_var(f"alpha1_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for b in B}
    alpha2 = {b: model.add_var(f"alpha2_{b}", obj=0, lb=0, ub=1, var_type=mip.CONTINUOUS) for b in B}
    
    for b in B:
        model.add_constr(mip.quicksum(A[a] * lambd1[a, b] for a in A) <= B[b] * y[b])
        model.add_constr(mip.quicksum(C[c] * lambd2[b, c] for c in C) <= B[b] * y[b])
        
        # flow constraint: model may become infeasable.
        model.add_constr(mip.quicksum(C[c] * lambd2[b, c] for c in C) == mip.quicksum(A[a] * lambd1[a, b] for a in A))
    
    for a in A:
        model.add_constr(mip.quicksum(lambd[a, b] for b in B) == 1)
    
    for a, b in product(A, B):

        # non-linear constraint `model.add_constr(lambd[a, b] == alpha[b] * x[a, b])` is remplaced by:
        model.add_constr(lambd[a, b] <= x[a, b])
        model.add_constr(lambd[a, b] <= alpha[b])
        model.add_constr(lambd[a, b] + 1 >= alpha[b] + x[a, b])
    
    for b in B:
        model.add_constr(mip.quicksum(x[a, b] for a in A) <= k)

    model.optimize()
    
    print(model.status)
    
    return {k for k, v in x.items() if v.x > 0}, {k for k, v in y.items() if v.x > 0}
